In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from mobilenet_models import create_mobilenetv3_large_model, create_mobilenetv3_large_model_opt_weights, create_mobilenetv3_large_model_opt
import wandb
from wandb.keras import WandbCallback
import os

2024-05-22 13:29:04.090628: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 13:29:04.090674: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 13:29:04.090714: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 13:29:04.100641: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../../../organized_dataset_v7/dataset/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="../../../organized_dataset_v7/dataset/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )

Found 430968 files belonging to 12 classes.


2024-05-22 13:29:43.549222: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31127 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5c:00.0, compute capability: 7.0


Found 37219 files belonging to 12 classes.


In [5]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_all/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=128,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_all/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=128,
    image_size=(224, 224)
    )

Found 964178 files belonging to 73 classes.


2024-05-16 17:44:45.147943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31127 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5c:00.0, compute capability: 7.0


Found 120495 files belonging to 73 classes.


In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_15k/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=128,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_15k/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=128,
    image_size=(224, 224)
    )

Found 312645 files belonging to 73 classes.


2024-05-20 20:19:09.663937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31125 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5c:00.0, compute capability: 7.0


Found 50568 files belonging to 73 classes.


# MobileNetv3 Large

In [3]:
# initialize wandb
wandb.init(project="MobileNet", config={"learning_rate": 0.001, "epochs": 30, "verbose": 1,"name": "MobileNetV3Large", "architecture": "MobileNetV3Large"})

# configs
cfg = wandb.config

# create the model
model = create_mobilenetv3_large_model(len(train_ds.class_names))

wandb: Currently logged in as: katrinalie (katrinamaster). Use `wandb login --relogin` to force relogin


In [4]:
# list of callbacks
callbacks = [
            WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.00001),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
          ]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [5]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=32, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

Epoch 1/30


2024-05-05 16:48:41.971596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-05-05 16:48:43.330020: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1e04fb7610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-05 16:48:43.330056: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2024-05-05 16:48:43.335967: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-05 16:48:43.429527: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


29644/29644 [==============================] - ETA: 0s - loss: 0.6991 - accuracy: 0.7934INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best)... Done. 0.1s


29644/29644 [==============================] - 765s 26ms/step - loss: 0.6991 - accuracy: 0.7934 - val_loss: 0.5135 - val_accuracy: 0.8456 - lr: 0.0010
Epoch 2/30
29644/29644 [==============================] - 739s 25ms/step - loss: 0.6529 - accuracy: 0.8120 - val_loss: 0.5152 - val_accuracy: 0.8499 - lr: 0.0010
Epoch 3/30
29644/29644 [==============================] - 736s 25ms/step - loss: 0.6582 - accuracy: 0.8148 - val_loss: 0.5380 - val_accuracy: 0.8480 - lr: 0.0010
Epoch 4/30
29644/29644 [==============================] - 744s 25ms/step - loss: 0.6698 - accuracy: 0.8163 - val_loss: 0.5485 - val_accuracy: 0.8462 - lr: 0.0010
Epoch 5/30
29644/29644 [==============================] - ETA: 0s - loss: 0.5691 - accuracy: 0.8400INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets


...

29644/29644 [==============================] - 717s 24ms/step - loss: 0.5272 - accuracy: 0.8496 - val_loss: 0.4821 - val_accuracy: 0.8679 - lr: 1.0000e-04
Epoch 9/30
29642/29644 [============================>.] - ETA: 0s - loss: 0.5201 - accuracy: 0.8514INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best)... Done. 0.1s


29644/29644 [==============================] - 725s 24ms/step - loss: 0.5201 - accuracy: 0.8514 - val_loss: 0.4820 - val_accuracy: 0.8687 - lr: 1.0000e-04
Epoch 10/30
29643/29644 [============================>.] - ETA: 0s - loss: 0.5169 - accuracy: 0.8520INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240505_163846-j4n63m90/files/model-best)... Done. 0.1s


29644/29644 [==============================] - 729s 25ms/step - loss: 0.5170 - accuracy: 0.8520 - val_loss: 0.4811 - val_accuracy: 0.8693 - lr: 1.0000e-04
Epoch 11/30
29644/29644 [==============================] - 742s 25ms/step - loss: 0.5119 - accuracy: 0.8533 - val_loss: 0.4835 - val_accuracy: 0.8692 - lr: 1.0000e-04
Epoch 12/30
29644/29644 [==============================] - 740s 25ms/step - loss: 0.5092 - accuracy: 0.8537 - val_loss: 0.4845 - val_accuracy: 0.8699 - lr: 1.0000e-04
Epoch 13/30
29644/29644 [==============================] - 736s 25ms/step - loss: 0.5066 - accuracy: 0.8545 - val_loss: 0.4840 - val_accuracy: 0.8703 - lr: 1.0000e-04
Epoch 14/30
29644/29644 [==============================] - 729s 25ms/step - loss: 0.4935 - accuracy: 0.8571 - val_loss: 0.4819 - val_accuracy: 0.8715 - lr: 1.0000e-05
Epoch 15/30
29644/29644 [==============================] - 736s 25ms/step - loss: 0.4901 - accuracy: 0.8578 - val_loss: 0.4817 - val_accuracy: 0.8721 - lr: 1.0000e-05


In [6]:
wandb.finish()

accuracy,▁▃▃▃▆▇▇▇▇▇█████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▇▇▄▃▂▂▂▂▂▂▂▁▁
val_accuracy,▁▂▂▁▆▆▇▇▇▇▇▇███
val_loss,▄▅▇█▂▂▁▁▁▁▁▁▁▁▁
accuracy,0.85785
best_epoch,9
best_val_loss,0.48111
epoch,14
loss,0.49007
val_accuracy,0.87214


# MobileNetv3 Large Optimized

In [3]:
# initialize wandb
wandb.init(project="MobileNet", config={"learning_rate": 0.001, "epochs": 30, "verbose": 1,"name": "MobileNetV3LargeOpt", "architecture": "MobileNetV3LargeOpt"})

# configs
cfg = wandb.config

# create the model
model = create_mobilenetv3_large_model_opt(len(train_ds.class_names))

wandb: Currently logged in as: katrinalie (katrinamaster). Use `wandb login --relogin` to force relogin


In [5]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=128, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

Epoch 1/30


2024-05-20 20:19:41.736734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-05-20 20:19:43.587237: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d6872545f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-20 20:19:43.587278: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2024-05-20 20:19:43.593343: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-20 20:19:43.680468: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2443/2443 [==============================] - ETA: 0s - loss: 0.9848 - accuracy: 0.7319INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best)... Done. 0.1s


2443/2443 [==============================] - 219s 87ms/step - loss: 0.9848 - accuracy: 0.7319 - val_loss: 0.6492 - val_accuracy: 0.8030 - lr: 0.0010
Epoch 2/30
2442/2443 [============================>.] - ETA: 0s - loss: 0.6602 - accuracy: 0.8131INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets


...

2443/2443 [==============================] - 211s 86ms/step - loss: 0.6602 - accuracy: 0.8131 - val_loss: 0.6029 - val_accuracy: 0.8191 - lr: 0.0010
Epoch 3/30
2442/2443 [============================>.] - ETA: 0s - loss: 0.5601 - accuracy: 0.8391INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best)... Done. 0.1s


2443/2443 [==============================] - 211s 86ms/step - loss: 0.5601 - accuracy: 0.8391 - val_loss: 0.5890 - val_accuracy: 0.8256 - lr: 0.0010
Epoch 4/30
2443/2443 [==============================] - 195s 80ms/step - loss: 0.4962 - accuracy: 0.8553 - val_loss: 0.6117 - val_accuracy: 0.8266 - lr: 0.0010
Epoch 5/30
2443/2443 [==============================] - 195s 80ms/step - loss: 0.4442 - accuracy: 0.8693 - val_loss: 0.6022 - val_accuracy: 0.8311 - lr: 0.0010
Epoch 6/30
2443/2443 [==============================] - 194s 79ms/step - loss: 0.4049 - accuracy: 0.8797 - val_loss: 0.6101 - val_accuracy: 0.8312 - lr: 0.0010
Epoch 7/30
2442/2443 [============================>.] - ETA: 0s - loss: 0.2909 - accuracy: 0.9112INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240520_201924-vnoodimm/files/model-best)... Done. 0.1s


2443/2443 [==============================] - 207s 85ms/step - loss: 0.2909 - accuracy: 0.9112 - val_loss: 0.5779 - val_accuracy: 0.8534 - lr: 1.0000e-04
Epoch 8/30
2443/2443 [==============================] - 193s 79ms/step - loss: 0.2443 - accuracy: 0.9238 - val_loss: 0.5909 - val_accuracy: 0.8554 - lr: 1.0000e-04
Epoch 9/30
2443/2443 [==============================] - 190s 78ms/step - loss: 0.2192 - accuracy: 0.9310 - val_loss: 0.6128 - val_accuracy: 0.8540 - lr: 1.0000e-04
Epoch 10/30
2443/2443 [==============================] - 188s 77ms/step - loss: 0.1994 - accuracy: 0.9364 - val_loss: 0.6352 - val_accuracy: 0.8535 - lr: 1.0000e-04
Epoch 11/30
2443/2443 [==============================] - 185s 76ms/step - loss: 0.1781 - accuracy: 0.9429 - val_loss: 0.6316 - val_accuracy: 0.8550 - lr: 1.0000e-05
Epoch 12/30
2443/2443 [==============================] - 186s 76ms/step - loss: 0.1740 - accuracy: 0.9442 - val_loss: 0.6344 - val_accuracy: 0.8553 - lr: 1.0000e-05


In [ ]:
wandb.save('model.h5)

In [6]:
wandb.finish()

accuracy,▁▄▅▅▆▆▇▇████
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▅▄▄▃▃▂▂▁▁▁▁
val_accuracy,▁▃▄▄▅▅██████
val_loss,█▃▂▄▃▄▁▂▄▇▆▇
accuracy,0.94422
best_epoch,6
best_val_loss,0.57787
epoch,11
loss,0.17398
val_accuracy,0.85526


# MobileNetv3 Large Optimized Made Weights

In [3]:
# initialize wandb
wandb.init(project="MobileNet", config={"learning_rate": 0.001, "epochs": 30, "verbose": 1,"name": "MobileNetV3LargeOpt", "architecture": "MobileNetV3LargeOpt"})

# configs
cfg = wandb.config

# create the model
model = create_mobilenetv3_large_model_opt_weights(len(train_ds.class_names))

wandb: Currently logged in as: katrinalie (katrinamaster). Use `wandb login --relogin` to force relogin


In [5]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=32, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

Epoch 1/30


2024-05-22 13:30:09.848422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-05-22 13:30:11.104426: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8194cc12c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-22 13:30:11.104457: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2024-05-22 13:30:11.110475: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-22 13:30:11.198430: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13468/13468 [==============================] - ETA: 0s - loss: 0.3156 - accuracy: 0.9033INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best)... Done. 0.1s


13468/13468 [==============================] - 342s 25ms/step - loss: 0.3156 - accuracy: 0.9033 - val_loss: 0.1760 - val_accuracy: 0.9492 - lr: 0.0010
Epoch 2/30
13468/13468 [==============================] - ETA: 0s - loss: 0.2317 - accuracy: 0.9282INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best)... Done. 0.1s


13468/13468 [==============================] - 340s 25ms/step - loss: 0.2317 - accuracy: 0.9282 - val_loss: 0.1620 - val_accuracy: 0.9539 - lr: 0.0010
Epoch 3/30
13468/13468 [==============================] - 322s 24ms/step - loss: 0.2080 - accuracy: 0.9356 - val_loss: 0.1679 - val_accuracy: 0.9534 - lr: 0.0010
Epoch 4/30
13468/13468 [==============================] - 321s 24ms/step - loss: 0.1920 - accuracy: 0.9404 - val_loss: 0.1660 - val_accuracy: 0.9556 - lr: 0.0010
Epoch 5/30
13468/13468 [==============================] - 324s 24ms/step - loss: 0.1805 - accuracy: 0.9440 - val_loss: 0.1622 - val_accuracy: 0.9587 - lr: 0.0010
Epoch 6/30
13467/13468 [============================>.] - ETA: 0s - loss: 0.1314 - accuracy: 0.9585INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/mobilenet/wandb/run-20240522_132955-flrylfvn/files/model-best)... Done. 0.1s


13468/13468 [==============================] - 332s 25ms/step - loss: 0.1314 - accuracy: 0.9585 - val_loss: 0.1586 - val_accuracy: 0.9601 - lr: 1.0000e-04
Epoch 7/30
13468/13468 [==============================] - 310s 23ms/step - loss: 0.1202 - accuracy: 0.9616 - val_loss: 0.1665 - val_accuracy: 0.9604 - lr: 1.0000e-04
Epoch 8/30
13468/13468 [==============================] - 293s 22ms/step - loss: 0.1124 - accuracy: 0.9634 - val_loss: 0.1754 - val_accuracy: 0.9603 - lr: 1.0000e-04
Epoch 9/30
13468/13468 [==============================] - 295s 22ms/step - loss: 0.1072 - accuracy: 0.9651 - val_loss: 0.1843 - val_accuracy: 0.9604 - lr: 1.0000e-04
Epoch 10/30
13468/13468 [==============================] - 296s 22ms/step - loss: 0.1001 - accuracy: 0.9670 - val_loss: 0.1847 - val_accuracy: 0.9609 - lr: 1.0000e-05
Epoch 11/30
13468/13468 [==============================] - 294s 22ms/step - loss: 0.0980 - accuracy: 0.9676 - val_loss: 0.1880 - val_accuracy: 0.9610 - lr: 1.0000e-05


In [6]:
wandb.finish()

accuracy,▁▄▄▅▅▇▇▇███
epoch,▁▂▂▃▄▅▅▆▇▇█
loss,█▅▅▄▄▂▂▁▁▁▁
val_accuracy,▁▅▅▅▆██████
val_loss,█▂▃▃▃▁▂▄▇▇▇
accuracy,0.97455
best_epoch,5
best_val_loss,0.15376
epoch,10
loss,0.07599
val_accuracy,0.96112
